# LangChain Chat Part 2 - Conversation

From DeepLearning.ai's `LangChain Chat with Your Data` course.

Tutorial: https://learn.deeplearning.ai/langchain-chat-with-your-data/lesson/7/chat

Be sure to persist vector store to disk by running langchain_chat_1_persist_vectore_store notebook before running this one.

In [1]:
import os
import openai
from dotenv import load_dotenv

load_dotenv()

openai.api_key  = os.environ['OPENAI_API_KEY']

## Load Vectore Store from Filesystem

In [2]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings

persist_directory = 'chroma/'
embedding = OpenAIEmbeddings()
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embedding)

## Build Memory

In [3]:
from langchain.memory import ConversationBufferMemory
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
retriever=vectordb.as_retriever()

# ConversationRetrievalChain adds a new step on top of the RetrievalQA chain.
# It takes the history and the new question, and condenses it into a standalone question to pass to the vector store to look up relevant documents.
# It also uses this condensed standalone question to pass to the language model to get an answer along with the relevant chunks from the vector store.
qa = ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory
)

In [5]:
question = "Is probability a class topic?"
result = qa({"question": question})
result["answer"]

'Yes, probability is a topic that will be covered in the class. The instructor mentions assuming familiarity with basic probability and statistics, and also mentions using a probabilistic interpretation to derive a learning algorithm.'

In [6]:
question = "why are those prerequesites needed?"
result = qa({"question": question})
result['answer']

'It is necessary to assume familiarity with basic probability and statistics because learning algorithms often rely on probabilistic interpretations to derive their algorithms. By understanding probability and statistics, we can make informed decisions about how to model and analyze data, and how to make predictions based on that data. The probabilistic interpretation allows us to quantify uncertainty and make predictions with confidence intervals. Without this understanding, it would be difficult to design effective learning algorithms that can accurately learn from data and make predictions.'